In [78]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [79]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/NLP Project/training_twitter_x_y_train.csv") 
train.head()

In [ ]:
train.info()

In [82]:
drop_cols = ['airline_sentiment_gold','name','tweet_id', 'retweet_count','tweet_created','user_timezone','tweet_coord','tweet_location' , 'airline' , 'negativereason_gold' ]
train.drop(drop_cols, axis = 1, inplace=True)

In [83]:
stops = stopwords.words('english')
stops += list(punctuation)
stops += ['flight','airline','flights','AA']

In [84]:
corpus = []
import re
for i in range(0 , len(train)):
  review = re.sub('[^a-zA-Z]' , ' ' , train['text'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stops]
  review = ' '.join(review)
  corpus.append(review)

In [85]:
v = TfidfVectorizer(analyzer='word', max_features=3150, max_df = 0.8, ngram_range=(1,1))
X = v.fit_transform(corpus).toarray()
y = train['airline_sentiment']

In [86]:
y.head()

0    negative
1    positive
2    positive
3    negative
4    negative
Name: airline_sentiment, dtype: object

In [87]:
from sklearn.preprocessing import LabelEncoder

In [88]:
y = LabelEncoder().fit_transform(y)

In [89]:
y[:20]

array([0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0])

In [90]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size =  0.2 , random_state = 0)

In [91]:
from sklearn.naive_bayes import MultinomialNB
senti_model = MultinomialNB().fit(X_train, y_train)

In [92]:
y_pred = senti_model.predict(X_test)

In [93]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test , y_pred)

array([[1359,   17,    5],
       [ 299,  132,   26],
       [ 216,   22,  120]])

In [94]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test , y_pred)

0.7336065573770492

In [76]:
clf = SVC(kernel="linear", C= 0.96 , gamma = 'scale')
clf.fit(X_train, y_train)
accuracy_score(y_test , clf.predict(X_test))

0.7773224043715847